### Creating a database

The first step is to create a database. We can use the connect function for this task. It is used for connecting to a database and if the database does not exist, the connect function also creates it.

In [8]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Connect to the database

conn = sqlite3.connect('sample.sqlite')

In [9]:
# The next step is to create a database cursor, which is required for executing SQL statements and fetching results.
cursor = conn.cursor()

In [10]:
# We can see the tables in this database using the following query and the fetchall function. We are basically querying the built-in sqlite_master table.

check_tables = "SELECT name FROM sqlite_master WHERE type='table';"
cursor.execute(check_tables)
print(cursor.fetchall())


[('products',)]


In [4]:
# We can see the tables in this database using the following query and the fetchall function. We are basically querying the built-in sqlite_master table.

### Creating tables

In [11]:
# Let’s create a table. We will write the query to create a table and then use the cursor to execute it.

create_products_table = """CREATE TABLE IF NOT EXISTS products (
                                    id integer PRIMARY KEY,
                                    category text NOT NULL,
                                    price real,
                                    cost real
                                );""" 

cursor.execute(create_products_table)

In [12]:
# Let’s check the tables in the sample database again.

check_tables = "SELECT name FROM sqlite_master WHERE type='table';"
cursor.execute(check_tables)
print(cursor.fetchall())


[('products',)]


We now see the products table exists in the database.

We can execute any query using the cursor. For instance, the following code snippet executes the query to select all the rows in the products table and assigns the result to a variable called result. Then, the fetchall function is called on the result variable to see what it contains.

In [13]:
result = cursor.execute("SELECT * FROM products;")
print(result.fetchall())


[]


In [8]:
# It does not return anything since the products table is empty. Let’s populate it using the insert into statement.

In [14]:
populate_products_table = """INSERT INTO products VALUES
                            (1001, "A",15.9,12.9),
                            (1002, "B",24.9,20.5),
                            (1003, "A",13.5,10.6),
                            (1004, "A",17.5,13.5),
                            (1005, "B",28.9,23.5)
                            ;"""

cursor.execute(populate_products_table)

                        

In [15]:
# show the results
result = cursor.execute("SELECT * FROM products;")
result.fetchall()

[(1001, 'A', 15.9, 12.9),
 (1002, 'B', 24.9, 20.5),
 (1003, 'A', 13.5, 10.6),
 (1004, 'A', 17.5, 13.5),
 (1005, 'B', 28.9, 23.5)]

In [16]:
# Note: The insert statement implicitly opens a transaction, which needs to be committed before changes are saved in the database.

In [17]:
# The products table should have 5 rows now. Let’s confirm by executing the select statement above.
# print the result line by line

result = cursor.execute("SELECT * FROM products;")
for row in result:
    print(row)


(1001, 'A', 15.9, 12.9)
(1002, 'B', 24.9, 20.5)
(1003, 'A', 13.5, 10.6)
(1004, 'A', 17.5, 13.5)
(1005, 'B', 28.9, 23.5)


In [18]:
# or print the result as a list

result = cursor.execute("SELECT * FROM products;")
print(result.fetchall())


[(1001, 'A', 15.9, 12.9), (1002, 'B', 24.9, 20.5), (1003, 'A', 13.5, 10.6), (1004, 'A', 17.5, 13.5), (1005, 'B', 28.9, 23.5)]


In [19]:
# We can, of course, write more advanced queries. For instance, the following query returns the rows that belong to category A.

result = cursor.execute("SELECT * FROM products WHERE category = 'A';")
print(result.fetchall())

[(1001, 'A', 15.9, 12.9), (1003, 'A', 13.5, 10.6), (1004, 'A', 17.5, 13.5)]


### One line of code to create a Pandas DataFrame

One of the nice things about sqlite3 module is that it is compatible with Pandas. Hence, we can easily write the results of a query into a Pandas DataFrame.

We can call the read_sql_query function of Pandas with the query and connection object.

In [20]:
products = pd.read_sql_query("SELECT * FROM products;", conn)
print(products)

     id category  price  cost
0  1001        A   15.9  12.9
1  1002        B   24.9  20.5
2  1003        A   13.5  10.6
3  1004        A   17.5  13.5
4  1005        B   28.9  23.5


In [ ]:
# Let’s create another table and populate it.

In [21]:
create_customers_table = """CREATE TABLE IF NOT EXISTS customers (
                                    id integer PRIMARY KEY,
                                    customer_id integer NOT NULL,
                                    ismember integer NOT NULL,
                                    purchase_date text,
                                    purchased_product integer,
                                    purchase_quantity integer
                                );"""

populate_customer_table = """INSERT INTO customers VALUES
                                    (1, 110, 0, "2022-12-23", 1002, 5),
                                    (2, 112, 0, "2022-12-14", 1001, 4),
                                    (3, 113, 1, "2022-12-08", 1003, 6),
                                    (4, 113, 1, "2022-12-14", 1002, 4),
                                    (5, 114, 0, "2022-12-21", 1004, 10)
                                ;"""

cursor.execute(create_customers_table)
cursor.execute(populate_customer_table)

conn.commit()

In [22]:
# show the results
result = cursor.execute("SELECT * FROM customers;")
result.fetchall()


[(1, 110, 0, '2022-12-23', 1002, 5),
 (2, 112, 0, '2022-12-14', 1001, 4),
 (3, 113, 1, '2022-12-08', 1003, 6),
 (4, 113, 1, '2022-12-14', 1002, 4),
 (5, 114, 0, '2022-12-21', 1004, 10)]

In [23]:
customers = pd.read_sql_query("SELECT * FROM customers;", conn)
customers

,id,customer_id,ismember,purchase_date,purchased_product,purchase_quantity
0,1,110,0,2022-12-23,1002,5
1,2,112,0,2022-12-14,1001,4
2,3,113,1,2022-12-08,1003,6
3,4,113,1,2022-12-14,1002,4
4,5,114,0,2022-12-21,1004,10


### More complex queries

Let’s write a query that retrieves data from both products and customers tables. It will include a join statement.

In [25]:
query = '''
        SELECT
            customer_id,
            purchased_product,
            purchase_quantity
        FROM customers c
        LEFT JOIN products p ON c.purchased_product = p.id
        '''
cursor.execute(query)
cursor.fetchall()

[(110, 1002, 5),
 (112, 1001, 4),
 (113, 1003, 6),
 (113, 1002, 4),
 (114, 1004, 10)]

You can use the description attribute to see the column names in the output of the query:

In [26]:
cursor.description



(('customer_id', None, None, None, None, None, None),
 ('purchased_product', None, None, None, None, None, None),
 ('purchase_quantity', None, None, None, None, None, None))

In [27]:
# save the names in a Python list using list comprehension
col_names = [description[0] for description in cursor.description]
col_names

['customer_id', 'purchased_product', 'purchase_quantity']

### Closing the connection
We can use the close method to close the current connection to the database

In [28]:
conn.close()
